# Crowd Anomaly Detection - Dataset Experiments

This notebook explores various datasets for crowd analysis and anomaly detection.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Configuration
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

## Dataset Information

### Recommended Datasets:

1. **UCSD Pedestrian Dataset**
   - Anomaly detection in pedestrian walkways
   - http://www.svcl.ucsd.edu/projects/anomaly/dataset.htm

2. **ShanghaiTech Dataset**
   - Crowd counting and density estimation
   - https://github.com/desenzhou/ShanghaiTechDataset

3. **UCF-Crime Dataset**
   - Anomaly detection in surveillance videos
   - https://www.crcv.ucf.edu/projects/real-world/

4. **CUHK Avenue Dataset**
   - Abnormal event detection
   - http://www.cse.cuhk.edu.hk/leojia/projects/detectabnormal/dataset.html

## Load Sample Video

In [ ]:
# Load a sample video
video_path = 'path/to/sample/video.mp4'

cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()

if ret:
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.title('Sample Frame')
    plt.axis('off')
    plt.show()

cap.release()

## Test YOLOv8 Detection

In [ ]:
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO('yolov8n.pt')

# Run inference
results = model(frame)

# Visualize results
annotated = results[0].plot()
plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
plt.title('YOLOv8 Detections')
plt.axis('off')
plt.show()

## Crowd Density Analysis

In [ ]:
# Extract person detections
person_boxes = []
for result in results:
    boxes = result.boxes
    for box in boxes:
        if int(box.cls[0]) == 0:  # person class
            person_boxes.append(box.xyxy[0].cpu().numpy())

print(f"Detected {len(person_boxes)} people")

# Visualize density heatmap
density_map = np.zeros(frame.shape[:2], dtype=np.float32)

for box in person_boxes:
    x1, y1, x2, y2 = map(int, box)
    cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
    cv2.circle(density_map, (cx, cy), 50, 1, -1)

density_map = cv2.GaussianBlur(density_map, (51, 51), 0)

plt.imshow(density_map, cmap='hot')
plt.colorbar(label='Density')
plt.title('Crowd Density Heatmap')
plt.axis('off')
plt.show()

## Pose Estimation for Activity Analysis

In [ ]:
import mediapipe as mp

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Process first person
if len(person_boxes) > 0:
    x1, y1, x2, y2 = map(int, person_boxes[0])
    person_crop = frame[y1:y2, x1:x2]
    
    rgb_crop = cv2.cvtColor(person_crop, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_crop)
    
    if results.pose_landmarks:
        # Draw landmarks
        annotated_crop = person_crop.copy()
        mp.solutions.drawing_utils.draw_landmarks(
            annotated_crop,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS
        )
        
        plt.imshow(cv2.cvtColor(annotated_crop, cv2.COLOR_BGR2RGB))
        plt.title('Pose Estimation')
        plt.axis('off')
        plt.show()

## Tracking Experiment

In [ ]:
# Test tracking over multiple frames
from collections import defaultdict

cap = cv2.VideoCapture(video_path)
track_history = defaultdict(list)

frame_count = 0
max_frames = 100

while cap.isOpened() and frame_count < max_frames:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Track with YOLOv8 built-in tracker
    results = model.track(frame, persist=True)
    
    if results[0].boxes.id is not None:
        boxes = results[0].boxes.xyxy.cpu().numpy()
        track_ids = results[0].boxes.id.cpu().numpy().astype(int)
        
        for box, track_id in zip(boxes, track_ids):
            x1, y1, x2, y2 = box
            cx, cy = (x1 + x2) / 2, (y1 + y2) / 2
            track_history[track_id].append((cx, cy))
    
    frame_count += 1

cap.release()

# Visualize tracks
plt.figure(figsize=(12, 8))
for track_id, positions in track_history.items():
    positions = np.array(positions)
    plt.plot(positions[:, 0], positions[:, 1], marker='o', label=f'Track {track_id}')

plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.title('Object Trajectories')
plt.legend()
plt.gca().invert_yaxis()
plt.show()

## Summary

This notebook demonstrates:
- Loading and processing video data
- Person detection with YOLOv8
- Crowd density estimation
- Pose estimation for activity analysis
- Multi-object tracking

Use these techniques as building blocks for the full anomaly detection system.